# **MÓDULO 27 - Projeto de Doenças Cardiovasculares - Regressão Logística**


Assim como na aula que trabalhamos com uma base de dados nova, com um contexto de modelo de propensão a compra de carros, para a atividade de vocês achei interessante trazer também novos desafios.

Nessa tarefa iremos construir um modelo que nos ajude a prever doenças cardiovasculares, a base contém dados reais.

age - idade dos pacientes

gender - genero (2 mulheres) (1 homens)

height - altura dos pacientes

weight - peso dos pacientes

gluc - glicose

smoke - fumante (1) não fumante (0)

alco - consume alcool (1) não consome (0)

active - realiza atividades fisicas (1) não realiza (0)

cardio_disease - tem doença cardio (1) não tem (0) - Variável target


Seu objetivo é utilizar esses dados históricos dos pacientes e construir um bom modelo de regressão capaz de indicar se novos pacientes estão propensos a doenças cariovasculares ou não.

In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, classification_report

# 1) Comece carregando e tratando a base de dados.
Assim como na aula essa nova base não passou por pré processamento nenhum então nessa etapa, carrega os dados, verifique os tipos de dados, verifique se temos dados faltantes e outliers.
Quando necessário realize o tratamento.


In [40]:
df = pd.read_csv("C:/Users/yurid/Downloads/CARDIO_BASE.csv", delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             10000 non-null  int64 
 1   gender          10000 non-null  int64 
 2   height          10000 non-null  int64 
 3   weight          10000 non-null  object
 4   cholesterol     10000 non-null  int64 
 5   gluc            10000 non-null  int64 
 6   smoke           10000 non-null  int64 
 7   alco            10000 non-null  int64 
 8   active          10000 non-null  int64 
 9   cardio_disease  10000 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 781.4+ KB


In [41]:
#Substituindo virgular por pontos e transformando object em float
df.replace(r'\,', '.', regex = True, inplace = True)
df['weight'] = df['weight'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             10000 non-null  int64  
 1   gender          10000 non-null  int64  
 2   height          10000 non-null  int64  
 3   weight          10000 non-null  float64
 4   cholesterol     10000 non-null  int64  
 5   gluc            10000 non-null  int64  
 6   smoke           10000 non-null  int64  
 7   alco            10000 non-null  int64  
 8   active          10000 non-null  int64  
 9   cardio_disease  10000 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 781.4 KB


In [42]:
df.describe()

,age,gender,height,weight,cholesterol,gluc,smoke,alco,active,cardio_disease
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,53.288300,1.345400,164.308200,74.303710,1.365000,1.222200,0.089000,0.053700,0.797200,0.503100
std,6.796234,0.475522,8.178796,14.566353,0.677658,0.565561,0.284758,0.225436,0.402105,0.500015
min,30.000000,1.000000,70.000000,30.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,1.000000,159.000000,65.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,54.000000,1.000000,165.000000,72.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
75%,58.000000,2.000000,170.000000,82.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,65.000000,2.000000,250.000000,200.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
fig = make_subplots(rows = 1, cols = 10)

for i, coluna in enumerate(df.columns):
    fig.add_traces(go.Histogram(x=df[coluna],
                      name = coluna),
               rows = 1,
               cols = i+1)
fig.update_layout(title_text = 'Histogramas da distribuição dos dados',
                  legend_title = 'Variável')
fig.show()


Conforme os histogramas e o df.describe(), podemos notar distribuições sem outliers, além de:
- Distribuições próximas à normal nas variáveis não-binárias (age, height, weight);
- Desbalanceamento entre as variáveis categóricas (gender, cholesterol, gluc, smoke e active);
- Balanceamento na variável target (cardio_disease)

# 2) Agora é hora de explorar os dados com uma análise bem completa.
Plote pelo menos 3 gráficos analisando o comportamento da variável cardio com outras variaveis da sua preferência (análise bivariada). Não se esqueça de trazer insights acerca do analisado.


In [91]:
#Traçando as métricas para os gráficos: médias para as variáveis não-binárias e contagem para binárias
med_idade_0 = df[df['cardio_disease'] == 0]['age'].mean()
med_idade_1 = df[df['cardio_disease'] == 1]['age'].mean()
med_height_0 = df[df['cardio_disease'] == 0]['height'].mean()
med_height_1 = df[df['cardio_disease'] == 1]['height'].mean()
med_weight_0 = df[df['cardio_disease'] == 0]['weight'].mean()
med_weight_1 =df[df['cardio_disease'] == 1]['weight'].mean()

medias = pd.DataFrame({'cardio_disease' : [0,1],
                       'age' :[med_idade_0,med_idade_1],
                       'height' :[med_height_0,med_height_1],
                       'weight' :[med_weight_0,med_weight_1]})


#Plotando os gráficos
fig = make_subplots(rows = 1,
                    cols = 3)

color = ['orchid', 'darkorange', 'tomato']

for i, coluna in enumerate(['age', 'height', 'weight']):
    fig.add_traces(go.Bar(x=medias['cardio_disease'],
                        y=medias[coluna],
                        name = coluna,
                        marker_color = color[i]),
                rows= 1,
                cols = i+1)

fig.update_layout(title_text = 'Relação entre a média da variável e cardio_disease',
                  legend_title = 'Variável')
fig.show()


Dos gráficos, temos que idade e peso mais avançados apresentam mais casos de doenças cardiovasculares, enquanto a altura não influencia nesse caso.

# 3) Nessa etapa você deve trazer a matriz de correlação e apontar insights acerca das variáveis com um relacionamento mais forte entre si.



In [122]:
corr = df.corr()

fig = go.Figure()

fig.add_traces(go.Heatmap(z=corr,
                          x= corr.index,
                          y= corr.columns,
                          colorscale='pinkyl',
                          texttemplate='%{z}'))

fig.update_layout(title_text = 'Matriz de correlação')

fig.show()

Da matriz de correlação, podemos notar que o maior valor está entre altura e gênero (49,49%), seguida de gênero e fumante (33,20%) e álcool e fumante (32,9%).
Todavia, no caso da variável target, temos apenas correlações fracas, ainda que as mais fortes sejam com idade (23,7&), colesterol (22,12%) e peso (18,33%). Assim, temos que idade, colesterol e peso são as maiores influenciadoras na aquisição de doenças cardiovasculares.

# 4) Essa é a sua última etapa pré modelo. Você deve:

A) Separar a base em treino e teste.

B) Você considera que essa base precisa que os dados sejam padronizados? Se sim, porque? Se acredita que devem, então realize essa etapa.

C) Verifique se os dados estão balanceados, se não, faça o balanceamento.


D) Visualize as bases de treino, teste (X E Y) e verifique se está tudo adequado.

É preciso padronizar os dados pois as escalas das variáveis são diferentes, de modo que é possível haver erros no modelo em virtude disso. Assim, cabe aplicar StandardScale para evitar erros.

In [133]:
#Separando em treino e teste e verificando seus tamanhos
x = df.drop('cardio_disease', axis = 1)
y= df[['cardio_disease']]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 42)
dataset = {'xtrain': xtrain, 'xtest': xtest, 'ytrain': ytrain, 'ytest': ytest}

for nome, variavel in dataset.items():
    print(f'Tamanho de {nome}: {variavel.shape}')

Tamanho de xtrain: (8000, 9)
Tamanho de xtest: (2000, 9)
Tamanho de ytrain: (8000, 1)
Tamanho de ytest: (2000, 1)


In [139]:
#Fazendo a padronização dos dados

sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

#Balanceando os dados
smote = SMOTE(random_state=42)
xtrain_bal, ytrain_bal = smote.fit_resample(xtrain, ytrain)

#Verificando
xtrain_bal

array([[ 0.10037973, -0.73138185, -1.25610705, ..., -0.31328402,
        -0.24037742,  0.50780078],
       [-0.19393504,  1.36727483,  0.08809078, ..., -0.31328402,
        -0.24037742,  0.50780078],
       [ 1.4247962 , -0.73138185, -0.27850863, ..., -0.31328402,
        -0.24037742, -1.96927621],
       ...,
       [ 0.98332404,  1.36727483,  0.65429181, ..., -0.31328402,
        -0.24037742,  0.50780078],
       [-1.92199261,  1.36727483,  1.88967284, ...,  3.19199173,
        -0.24037742,  0.50780078],
       [-1.07687935, -0.73138185,  0.84528083, ..., -0.31328402,
        -0.24037742,  0.50780078]], shape=(8064, 9))

In [140]:
ytrain_bal

,cardio_disease
0,0
1,0
2,1
3,1
4,0
...,...
8059,0
8060,0
8061,0
8062,0


In [142]:
xtest

array([[-0.61355981, -0.70657652, -0.05203742, ..., -0.30966177,
        -0.22941573, -2.0382932 ],
       [ 0.12212341,  1.41527489,  1.418985  , ..., -0.30966177,
        -0.22941573,  0.49060655],
       [ 0.12212341, -0.70657652, -0.78754862, ..., -0.30966177,
        -0.22941573,  0.49060655],
       ...,
       [-1.64351632, -0.70657652,  0.68347379, ..., -0.30966177,
        -0.22941573,  0.49060655],
       [-0.46642316, -0.70657652,  0.43830339, ..., -0.30966177,
        -0.22941573,  0.49060655],
       [ 1.29921657,  1.41527489, -0.05203742, ..., -0.30966177,
        -0.22941573,  0.49060655]], shape=(2000, 9))

In [143]:
ytest

,cardio_disease
6252,0
4684,0
1731,1
4742,0
4521,1
...,...
6412,1
8285,1
7853,0
1095,1


# 5) Realize a etapa de treinamento do modelo:

A) Faça o treinamento do modelo.

B) Traga o intercept e os coeficientes.

c) Avalie as métricas do modelo treinado

D) Justifique se te parece que o modelo tem feito boas previsões ou não.

In [146]:
#Treinando o modelo
logistic = LogisticRegression(random_state=0)
logistic.fit(xtrain_bal, ytrain_bal)

#Trazendo o intercept e os coeficientes
print(logistic.intercept_)
print(logistic.coef_)

#Avaliando as métricas do modelo
previsoes = logistic.predict(xtrain_bal)
relatorio = classification_report(ytrain_bal, previsoes)

print('Relatório de métricas: ')
print(relatorio)


[0.01480453]
[[ 0.44145949  0.00862079 -0.07862564  0.35096437  0.4081836  -0.07356343
  -0.02782602 -0.00872331 -0.07136147]]
Relatório de métricas: 
              precision    recall  f1-score   support

           0       0.63      0.66      0.64      4032
           1       0.64      0.61      0.62      4032

    accuracy                           0.64      8064
   macro avg       0.64      0.64      0.63      8064
weighted avg       0.64      0.64      0.63      8064



C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Conforme as métricas do relatório, temos:

- Acurácia global de 64%, que significa 64% de previsões corretas;
- Para a classe 0:
    - 63% de precisão, que denota que 63% das instâncias previstas como classe 0 realmente são da classe 0;
    - 66% de recall, que denota que 66% das instâncias da classe 0 foram previstas corretamente;
    - 64% de f1-score, que denota a média harmônica entre precisão e recall.



- Para a classe 1:
    - 64% de precisão, que denota que 64% das instâncias previstas como classe 1 realmente são da classe 1;
    - 61% de recall, que denota que 61% das instâncias da classe 1 foram previstas corretamente;
    - 62% de f1-score, que denota a média harmônica entre precisão e recall.


# 6) Teste seu modelo!

A) Aplique o modelo aos dados de teste.

B) Avalie as métricas do modelo treinado

C) Plote o gráfico da curva AUC-ROC e explique o que consegue analisar através do gráfico.

In [148]:
#Aplicando o modelo aos dados de teste
previsoes_teste = logistic.predict(xtest)

#Verificando as métricas
relatorio_teste = classification_report(ytest, previsoes_teste)

print('Relatório do teste: ')
print(relatorio_teste)

Relatório do teste: 
              precision    recall  f1-score   support

           0       0.65      0.69      0.67      1001
           1       0.67      0.62      0.64       999

    accuracy                           0.66      2000
   macro avg       0.66      0.66      0.66      2000
weighted avg       0.66      0.66      0.66      2000



Do relatório de teste, temos relativa melhora em relação ao modelo de treino, conforme os resultados abaixo:

- Acurácia global de 66%, que significa 66% de previsões corretas;
- Para a classe 0:
    - 65% de precisão, que denota que 65% das instâncias previstas como classe 0 realmente são da classe 0;
    - 69% de recall, que denota que 69% das instâncias da classe 0 foram previstas corretamente;
    - 67% de f1-score, que denota a média harmônica entre precisão e recall.



- Para a classe 1:
    - 67% de precisão, que denota que 67% das instâncias previstas como classe 1 realmente são da classe 1;
    - 62% de recall, que denota que 62% das instâncias da classe 1 foram previstas corretamente;
    - 64% de f1-score, que denota a média harmônica entre precisão e recall.

Logo, o modelo tem eficácia moderada para prever pacientes com doenças cardiovasculares a partir das variáveis em estudo.


In [151]:
fpr, tpr, thresholds = roc_curve(ytest, previsoes_teste)
curva_roc = roc_auc_score(ytest, previsoes_teste)

print('AUC: {:2f}'.format(curva_roc))

AUC: 0.656465


In [183]:
fig = go.Figure()

fig.add_traces(go.Scatter(x=fpr,
                          y=tpr,
                          name = 'Curva ROC-AUC',
                          marker_color = 'purple',
                          fill = 'tozeroy',
                          line = dict(shape = 'spline')))

fig.add_traces(go.Scatter(x=[1,0],
                          y=[1,0],
                          name = 'Limiar',
                          line = dict(dash='dash',
                                      color = 'tomato')))

fig.add_annotation(text=f'Área = {curva_roc}',
                   y= 0.2,
                   x = 0.9,
                   showarrow = False,
                   bordercolor='white')

fig.update_layout(title_text = 'Curva ROC-AUC',
                  xaxis_title = 'Taxa de falsos positivos',
                  yaxis_title = 'Taxa de verdadeiros positivos',)

fig.show()

Diante disso, temos que a área da curva ROC é 0,65, que significa que o modelo consegue separar os dados relativamente bem, ou seja, se pegarmos aleatoriamente um valor 0 e um valor 1, há 65% de chance de o modelo atribuir um valor maior de probabilidade para o valor 1.

# 7) Explique:

A) Explique com suas palavras regressão logistica.

B) Explique porque a regressão logistica é um modelo de classificação.

C) Explique quais pontos em comum a regressão logistica tem da regressão linear.

R: Regressão logística é um modelo estatístico utilizado para problemas de classificação binários em que prevemos a probabilidade de algo acontecer ou não (0, 1). Embora tenha valores de probabilidade como resultado, utiliza-se tais valores para atribuir um rótulo aos dados (0 ou 1) conforme o treinamento do modelo. Por fim, a regressão logística e regressão linear têm em comum a combinação linear, pois apresentam coeficientes (pesos) para cada variável de entrada.
